<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=46266ae0f8cfbdbbf94f38e4ddf53e3cf9f330eaf75a1c65d53c7ca2754467cb
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-09 14:48:34
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.41 C
-------------------
Today PnL: -1.48 L (-1.03%)
Current PnL: -27.89 L (-17.78%)
CY Booked + Current PnL: -13.60 L (-8.67%)
-------------------
Total profit:  1.66 L
Total loss:  -29.55 L
-------------------
Total Booked + Current PnL: 13.38 L (10.31%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.12%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 93.34 L (66.11%)
Deployed:  1.30 C
Current:  1.41 C
CAGR/XIRR %: 4.57%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.75,14.35,6.16,21.40,18697.0,38095.0,303521.0,6.07,59.0,X-LC,5.69,14.0,2.04,2.15,21.59,XY25,NTT,FMCG
77,TTKPRESTIG,770.00,0.51,-21.00,26.70,0.09,21257.0,-21162.0,79615.0,84.85,38.0,M-SC,3.30,253.0,-1.00,0.56,4.60,OX40N,NTT,DURABLES
17,COALINDIA,484.83,-1.53,2.98,15.78,19.23,24449.0,4478.0,154938.0,21.35,65.0,L-LC,7.05,182.0,0.18,1.10,24.88,XY25,ATH,MINING
39,INDIGOPNTS,1408.00,-1.33,-14.42,16.88,0.02,25216.0,-25176.0,149383.0,127.68,54.0,M-SC,8.07,240.0,-1.00,1.06,29.94,OX40N,NTT,PAINTS
49,MASFIN,398.61,-2.74,-4.33,27.58,22.05,25852.0,-4245.0,93735.0,-17.34,46.0,H-SC,6.47,168.0,-0.16,0.66,36.25,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,1.63,-17.82,52.73,25.51,80643.0,-33160.0,152935.0,-6.04,44.0,H-LC,1.74,50.0,-0.41,1.08,12.70,AR,ATH,ELECTRICAL
44,JCHAC,2282.00,1.36,-33.96,51.40,-0.01,43386.0,-43396.0,84409.0,17182.61,52.0,M-SC,13.73,235.0,-1.00,0.60,8.03,OX40N,BTT,AC
1,ABB,7934.00,1.32,-6.66,55.95,45.57,136630.0,-17419.0,244200.0,-40.03,45.0,H-MC,4.69,121.0,-0.13,1.73,3.91,AR,NTT,ELECTRICAL
32,HCLTECH,1922.01,0.91,7.84,15.67,24.73,40881.0,18963.0,260887.0,12.76,56.0,X-LC,6.25,13.0,0.46,1.85,24.27,X40,ATH,IT
36,ICICIPRULI,790.00,0.87,13.72,14.95,30.73,33495.0,27039.0,224044.0,-12.89,72.0,X-MC,6.62,79.0,0.81,1.59,28.24,X40,ATH,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,IEX,219.00,-7.18,-5.46,57.20,48.63,108133.0,-10910.0,189044.0,-37.19,48.0,H-SC,13.71,137.0,-0.10,1.34,5.53,XR,NTT,MISC
56,RAJOOENG,143.10,-4.65,-33.12,108.75,39.61,74548.0,-33950.0,68550.0,-54.69,50.0,H-SC,30.26,136.0,-0.46,0.49,10.96,AR,ATH,MISC
76,TRIDENT,48.00,-3.43,-29.91,83.35,28.51,53898.0,-27590.0,64665.0,-17.17,38.0,M-SC,7.43,226.0,-0.51,0.46,10.65,XR,NTT,TEXTILES
18,COFFEEDAY,80.00,-3.35,-42.93,137.04,35.27,88802.0,-48749.0,64800.0,-55.73,41.0,L-SC,4.14,268.0,-0.55,0.46,58.60,XR,NTT,HOTELS
4,ALKYLAMINE,4546.37,-3.14,-33.69,192.00,93.63,128546.0,-34012.0,66951.0,-29.06,41.0,H-SC,16.64,143.0,-0.26,0.47,1.22,SR,ATH,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.88,-2.62,113.51,107.91,163199.0,-3869.0,143775.0,-20.96,50.0,M-SC,10.12,220.0,-0.02,1.02,7.20,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.33,-14.42,16.88,0.02,25216.0,-25176.0,149383.0,127.68,54.0,M-SC,8.07,240.0,-1.00,1.06,29.94,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,-0.25,-17.82,60.66,32.03,65718.0,-23497.0,108338.0,-11.00,65.0,H-SC,6.89,157.0,-0.36,0.77,21.35,OX40N,NTT,IT
77,TTKPRESTIG,770.00,0.51,-21.00,26.70,0.09,21257.0,-21162.0,79615.0,84.85,38.0,M-SC,3.30,253.0,-1.00,0.56,4.60,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.37,-21.99,45.47,13.47,95649.0,-59310.0,210357.0,-67.92,49.0,H-SC,4.58,159.0,-0.62,1.49,6.96,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,-1.53,2.98,15.78,19.23,24449.0,4478.0,154938.0,21.35,65.0,L-LC,7.05,182.0,0.18,1.10,24.88,XY25,ATH,MINING
84,WIPRO,420.00,-0.10,7.44,60.34,72.27,109852.0,12607.0,182055.0,-7.89,50.0,M-LC,6.17,101.0,0.11,1.29,14.07,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.10,7.44,60.34,72.27,109852.0,12607.0,182055.0,-7.89,50.0,M-LC,6.17,101.0,0.11,1.29,14.07,XR,NTT,IT
25,FINCABLES,1641.55,-0.88,-2.62,113.51,107.91,163199.0,-3869.0,143775.0,-20.96,50.0,M-SC,10.12,220.0,-0.02,1.02,7.20,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-1.12,-7.92,124.51,106.72,141397.0,-9773.0,113563.0,-54.44,34.0,H-SC,7.40,138.0,-0.07,0.80,13.60,AR,ATH,MISC
37,IEX,219.00,-7.18,-5.46,57.20,48.63,108133.0,-10910.0,189044.0,-37.19,48.0,H-SC,13.71,137.0,-0.10,1.34,5.53,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,-1.32,-5.56,43.61,35.63,86316.0,-11646.0,197928.0,-18.95,36.0,H-MC,3.19,119.0,-0.13,1.40,10.19,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-1.13,-17.39,34.10,10.78,67696.0,-41796.0,198522.0,-50.96,12.0,X-LC,6.31,1.0,-0.62,1.41,0.00,X40,NTT,FMCG
7,AWL,485.00,-2.17,-27.76,117.59,57.19,277048.0,-90522.0,235605.0,-65.80,23.0,X-MC,9.28,58.0,-0.33,1.67,0.00,XY24,NTT,FMCG
52,PAGEIND,51605.07,-1.28,-13.13,50.36,30.62,103701.0,-31134.0,205920.0,-38.20,25.0,X-MC,16.43,57.0,-0.30,1.46,0.00,X40,ATH,APPARELS
8,BAJAJHFL,181.50,-1.77,-20.14,94.68,55.47,183955.0,-48995.0,194291.0,-29.32,32.0,X-MC,16.35,63.0,-0.27,1.38,0.00,X40N,ATH,FINANCE
78,UNITDSPR,1644.00,-0.83,-1.64,22.78,20.77,52464.0,-3836.0,230308.0,-10.18,32.0,X-MC,0.97,70.0,-0.07,1.63,5.17,X40N,NTT,BREWERIES


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.00,-0.83,-1.64,22.78,20.77,52464.0,-3836.0,230308.0,-10.18,32.0,X-MC,0.97,70.0,-0.07,1.63,5.17,X40N,NTT,BREWERIES
45,JIOFIN,387.00,-2.08,-7.54,34.63,24.48,85608.0,-20158.0,247207.0,-9.13,38.0,X-LC,1.69,37.0,-0.24,1.75,43.29,XY24,BTT,FINANCE
65,SIEMENS,4671.50,1.63,-17.82,52.73,25.51,80643.0,-33160.0,152935.0,-6.04,44.0,H-LC,1.74,50.0,-0.41,1.08,12.70,AR,ATH,ELECTRICAL
81,VBL,671.64,-2.00,-0.84,36.82,35.67,115318.0,-2648.0,313194.0,-12.64,54.0,X-LC,2.21,5.0,-0.02,2.22,12.90,X40N,ATH,FMCG
20,DABUR,735.00,0.21,1.77,41.20,43.70,104016.0,4399.0,252467.0,-6.68,65.0,X-MC,2.24,73.0,0.04,1.79,17.09,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-1.03,-28.32,129.01,64.15,220039.0,-67391.0,170560.0,-58.06,33.0,X-SC,6.28,84.0,-0.31,1.21,0.0,XY24,BTT,CEMENT
7,AWL,485.00,-2.17,-27.76,117.59,57.19,277048.0,-90522.0,235605.0,-65.80,23.0,X-MC,9.28,58.0,-0.33,1.67,0.0,XY24,NTT,FMCG
8,BAJAJHFL,181.50,-1.77,-20.14,94.68,55.47,183955.0,-48995.0,194291.0,-29.32,32.0,X-MC,16.35,63.0,-0.27,1.38,0.0,X40N,ATH,FINANCE
43,ITC,452.00,-1.13,-17.39,34.10,10.78,67696.0,-41796.0,198522.0,-50.96,12.0,X-LC,6.31,1.0,-0.62,1.41,0.0,X40,NTT,FMCG
52,PAGEIND,51605.07,-1.28,-13.13,50.36,30.62,103701.0,-31134.0,205920.0,-38.20,25.0,X-MC,16.43,57.0,-0.30,1.46,0.0,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-2.01,-28.01,98.41,42.84,46056.0,-18206.0,46800.0,-52.62,50.0,X-SC,27.03,83.0,-0.40,0.33,5.88,XY24,NTT,MISC
58,RELAXO,1176.00,-0.21,-48.32,194.70,52.31,146066.0,-70139.0,75021.0,-45.02,40.0,X-SC,7.82,92.0,-0.48,0.53,0.42,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.12,-40.12,125.79,35.20,96920.0,-51621.0,77049.0,1.08,33.0,X-SC,17.32,93.0,-0.53,0.55,0.12,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-1.67,-1.39,25.83,24.08,32336.0,-1764.0,125186.0,-9.34,47.0,X-SC,6.29,91.0,-0.05,0.89,17.95,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.50,-17.58,73.89,43.32,99190.0,-28632.0,134240.0,-29.17,44.0,X-SC,7.86,90.0,-0.29,0.95,3.07,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-1.13,-17.39,34.10,10.78,67696.0,-41796.0,198522.0,-50.96,12.0,X-LC,6.31,1.0,-0.62,1.41,0.00,X40,NTT,FMCG
75,TMPV,600.00,-1.54,-27.46,69.54,22.98,113945.0,-62042.0,163856.0,-24.30,41.0,X-LC,4.14,2.0,-0.54,1.16,3.06,XY24,NTT,AUTO
73,TCS,4389.97,-0.08,-11.97,37.13,20.71,123623.0,-45291.0,332946.0,-24.27,46.0,X-LC,2.48,3.0,-0.37,2.36,11.25,X40,ATH,IT
81,VBL,671.64,-2.00,-0.84,36.82,35.67,115318.0,-2648.0,313194.0,-12.64,54.0,X-LC,2.21,5.0,-0.02,2.22,12.90,X40N,ATH,FMCG
41,INFY,1972.00,-0.14,9.67,22.41,34.24,76898.0,30255.0,343143.0,-14.57,49.0,X-LC,7.66,6.0,0.39,2.43,18.75,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.48,-35.09,104.73,32.89,52484.0,-27092.0,50114.0,-719.08,56.0,L-MC,7.70,259.0,-0.52,0.35,38.01,XR,NTT,BANKS
13,BSOFT,831.70,-1.99,-23.30,96.60,50.79,104210.0,-32775.0,107878.0,-1.43,48.0,H-SC,10.62,171.0,-0.31,0.76,25.60,XR,ATH,IT
69,SURYODAY,216.00,-1.71,-18.83,51.57,23.03,74962.0,-33711.0,145360.0,57.89,53.0,H-SC,4.10,165.0,-0.45,1.03,44.23,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-1.33,-14.42,16.88,0.02,25216.0,-25176.0,149383.0,127.68,54.0,M-SC,8.07,240.0,-1.00,1.06,29.94,OX40N,NTT,PAINTS
17,COALINDIA,484.83,-1.53,2.98,15.78,19.23,24449.0,4478.0,154938.0,21.35,65.0,L-LC,7.05,182.0,0.18,1.10,24.88,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-2.43,-30.58,62.64,12.90,95009.0,-66822.0,151674.0,123.56,59.0,H-SC,10.26,173.0,-0.70,1.07,26.12,OX40N,NTT,FINANCE
72,TATAELXSI,9161.00,-0.25,-17.82,60.66,32.03,65718.0,-23497.0,108338.0,-11.00,65.0,H-SC,6.89,157.0,-0.36,0.77,21.35,OX40N,NTT,IT
17,COALINDIA,484.83,-1.53,2.98,15.78,19.23,24449.0,4478.0,154938.0,21.35,65.0,L-LC,7.05,182.0,0.18,1.10,24.88,XY25,ATH,MINING
36,ICICIPRULI,790.00,0.87,13.72,14.95,30.73,33495.0,27039.0,224044.0,-12.89,72.0,X-MC,6.62,79.0,0.81,1.59,28.24,X40,ATH,INSURANCE
82,VOLTAS,1856.00,0.04,14.97,26.28,45.20,57936.0,28713.0,220455.0,3.76,63.0,X-MC,4.91,80.0,0.50,1.56,22.79,XY25,NTT,AC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.31
1,25,44.40
2,50,76.47


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.76
MC    30.75
LC    26.51
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.37
X40      24.00
X40N     12.80
XY25     10.71
XR        8.93
AR        8.57
OX40N     7.69
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.38
X-LC    23.04
H-SC    22.83
M-SC    11.31
X-SC     7.84
H-MC     4.60
M-MC     1.42
M-LC     1.29
L-LC     1.10
H-LC     1.08
L-SC     0.78
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.10,-9.20,45.03
IT,12.89,-17.51,76.78
FINANCE,11.08,-13.95,61.78
MISC,7.07,-30.21,83.47
ELECTRICAL,5.94,-11.76,52.70
PAINTS,5.76,-13.63,30.51
INSURANCE,4.75,-1.25,35.89
PHARMA,3.95,-5.48,38.68
AUTO,2.78,-33.61,79.93


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3248030.0,21
AR,1361578.0,10
XR,1229773.0,12
X40,1135867.0,15
X40N,935291.0,9
OX40N,707804.0,10
XY25,430396.0,7
SR,285568.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3568936.0,24
X-MC,1593438.0,16
M-SC,1464116.0,15
X-LC,972914.0,13
X-SC,812893.0,8
H-MC,428525.0,3
L-SC,181395.0,2
M-LC,109852.0,1
H-LC,80643.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1251764.0      6
           AR         926758.0      5
M-SC       XY24       843885.0      6
H-SC       XR         794390.0      7
X-MC       X40        514860.0      7
           XY24       448818.0      3
X-LC       X40        425419.0      6
X-MC       X40N       404575.0      4
X-SC       X40N       318874.0      3
H-SC       OX40N      310456.0      4
M-SC       OX40N      304755.0      5
X-SC       XY24       298431.0      3
H-SC       SR         285568.0      2
X-MC       XY25       225185.0      2
H-MC       AR         222946.0      2
X-LC       X40N       211842.0      2
H-MC       XY24       205579.0      1
X-LC       XY24       199553.0      2
X-SC       X40        195588.0      2
M-SC       XR         184245.0      2
X-LC       XY25       136100.0      3
M-SC       AR         131231.0      2
M-LC       XR         109852.0      1
L-SC       OX40N       92593.0      1
           XR          88802.0      1
H-LC       AR          80643.0      1
L-MC       XR          52484.0      1
M-MC       XY25        44662.0      1
L-LC       XY25        24449.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 53.0 seconds
